In [32]:
from collections import Counter
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np
from collections import defaultdict
import pandas as pd
import gzip

In [25]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Sports_and_Outdoors_5.json.gz')


KeyboardInterrupt: 

In [7]:
df.shape

(296337, 9)

In [10]:
import pickle
df.to_pickle('reviews_Sports_and_Outdoors_5.h5')

In [11]:
print df[:2]

       reviewerID        asin     reviewerName helpful  unixReviewTime  \
0    AIXZKN4ACSKI  1881509818     David Briner  [0, 0]      1390694400   
1  A1L5P841VIO02V  1881509818  Jason A. Kramer  [1, 1]      1328140800   

                                          reviewText  overall   reviewTime  \
0  This came in on time and I am veru happy with ...      5.0  01 26, 2014   
1  I had a factory Glock tool that I was using fo...      5.0   02 2, 2012   

                             summary  
0                     Woks very good  
1  Works as well as the factory tool  


In [2]:
from sklearn.utils import shuffle
df = pd.read_pickle('reviews_Sports_and_Outdoors_5.h5')
df.to_csv('reviews_Sports_and_Outdoors_5.csv')
df = shuffle(df)

In [14]:
from sklearn.model_selection import train_test_split

Training_DataFrame, Testing_DataFrame = train_test_split(df, test_size=0.2)

In [15]:
print Training_DataFrame.shape
print Testing_DataFrame.shape

(237069, 9)
(59268, 9)


In [5]:
print "Training dataset details:"
Uniq_movies_Train =  len(set(Training_DataFrame['asin'].values))
print "Number of item:", Uniq_movies_Train
Uniq_users_Train =   len(set(Training_DataFrame['reviewerID'].values))
print "Number of Users:", Uniq_users_Train
Uniq_ratings_Train = len(set(Training_DataFrame['overall'].values))
print "Number of ratings:", Uniq_ratings_Train
print "\n"

Training dataset details:
Number of item: 18355
Number of Users: 35590
Number of ratings: 5




In [23]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(Training_DataFrame[['reviewerID', 'asin', 'overall']], reader)
data_train = data.build_full_trainset()

In [24]:
algo = SVD()
algo.fit(data_train)

In [25]:
data_test = Dataset.load_from_df(Testing_DataFrame[['reviewerID', 'asin', 'overall']], reader)
testset = data_test.build_full_trainset().build_testset()

predictions = algo.test(testset)

In [26]:
print predictions[:10]

[Prediction(uid='A36EH00ULIZ29N', iid='B0019MNKX2', r_ui=4.0, est=4.440124442519867, details={u'was_impossible': False}), Prediction(uid='A1W2M2G4DG1IP8', iid='B00453MQXC', r_ui=3.0, est=4.1088852584532285, details={u'was_impossible': False}), Prediction(uid='A2WB7LZ595CR50', iid='B001K72L9K', r_ui=4.0, est=3.922118377910566, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W', iid='B003CI81NA', r_ui=5.0, est=4.603559539073942, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W', iid='B0007XY8TY', r_ui=5.0, est=4.533390567687502, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W', iid='B00BNPV1RW', r_ui=5.0, est=4.543067932365405, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W', iid='B0014VTREO', r_ui=3.0, est=4.704861167129625, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W', iid='B000NOSISO', r_ui=5.0, est=4.748675668577832, details={u'was_impossible': False}), Prediction(uid='A18RX4S2ZIL84W

In [27]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.9318
MAE:  0.6774


0.6774112408105691

In [33]:
def Recommeder(user, predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if uid == user:
            top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [36]:
Recommeder(u'A1W2M2G4DG1IP8', predictions)

defaultdict(list, {'A1W2M2G4DG1IP8': [('B00453MQXC', 4.1088852584532285)]})